In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
pathToRfmixResults = "/mnt/disks/local_ancestry_analysis/ukb_rfmix_results_matteo/"
outputPath = "/mnt/disks/local_ancestry_analysis/ukb_tractor_analysis/"

In [3]:
# Take sample mapping and turn into list
cho2Matteo = pd.read_csv("cholab_to_matteo_sampleMapping.txt", delim_whitespace=True)
cho2Matteo.head(5)
print(cho2Matteo.dtypes)

cholab    int64
matteo    int64
dtype: object


In [4]:
print(cho2Matteo.shape)
cho2Matteo = cho2Matteo[(cho2Matteo['matteo'] >= 0) & (cho2Matteo['cholab'] >= 0)]
print(cho2Matteo.shape)

(488377, 2)
(488172, 2)


In [5]:
sampleMap = dict(zip(cho2Matteo.matteo, cho2Matteo.cholab))

In [6]:
# Function to recode haplotype sample names from msp.tsv file
def recodeSamples(sampleMap, admixed_samples_genopheno_matched, samplesList):
    samples_cho = [] # Recoded samples
    samples_rfmix_idxs_to_read = [] # Indexes of samples (ie columns) in rfmix results that have matching genotype data + phenotype
    samples_cho_matched = [] 
    

    for idx in range(len(samplesList)):
        sample = samplesList[idx]
        sample_matteo = int(sample.split('_')[0])
        sample_cho = sampleMap.get(sample_matteo)
        
        if (sample_cho != None) and (sample_cho in admixed_samples_genopheno_matched):
            sample_string = str(sample_cho) + "_" + str(sample_cho)
            samples_cho.append(sample_string + ".0")
            samples_cho.append(sample_string + ".1")
            
            samples_cho_matched.append(sample_cho)
            
            # Store index of column mapping to sample id to read
            samples_rfmix_idxs_to_read.append(2 * idx + 6) # Offset 
            samples_rfmix_idxs_to_read.append(2 * idx + 6 + 1) 
            
            
    return samples_cho_matched, samples_cho, samples_rfmix_idxs_to_read

In [17]:
# Read in samples of admixed samples with no missing data (sample order as genotype files)
fname = "/mnt/disks/local_ancestry_analysis/ukb_tractor_analysis/input_data/ukb_hap_chr19_v2_admixed_final.psam"
admixed_samples_genopheno_matched =  pd.read_csv(fname, delim_whitespace=True).iloc[:, 0].values.tolist()
print(len(admixed_samples_genopheno_matched))
admixed_samples_genopheno_matched[:5] # Note: these are sample ids from our lab's sample files

2539


[5432249, 5093858, 3329090, 3640465, 4911551]

In [8]:
# Check column mismatch in msp files
for binIndex in range(1,3):
    fname = "pca.chr19.bin" + str(binIndex) + ".msp.tsv.gz"
    # Read in columns only
    tmp1 = pd.read_csv(pathToRfmixResults + fname , delim_whitespace=True, skiprows=1, nrows=1)
    tmp2 = pd.read_csv(pathToRfmixResults + fname , delim_whitespace=True, skiprows=2, nrows=1) # Correct columns
    
    print(len(tmp1.columns), len(tmp2.columns)) 

9979 9978
9999 9998


In [9]:
# Create dictionary of datatypes for data in msp file (haplotype data needs to be integers only)
def getDatatypes(sample_columns):
    rfmixResults_datatypes = dict.fromkeys(sample_columns, np.int8)
    rfmixResults_datatypes[0] = np.int8 #chm
    rfmixResults_datatypes[1] = np.int64 #spos
    rfmixResults_datatypes[2] = np.int64 #epos
    rfmixResults_datatypes[3] = np.float64 #sgpos
    rfmixResults_datatypes[4] = np.float64 #egpos
    rfmixResults_datatypes[5] = np.int64 #nsnps
    
    return rfmixResults_datatypes

In [18]:
# Map individuals from RFMIX results (Matteo back to our sample index mapping)
numberOfBins = 96 # partitions of UKB individuals
rfmixResults_admixedAfricanOnly = []
samples_cho_matched = []
for binIndex in range(1, numberOfBins + 1): 
    fname = "pca.chr19.bin" + str(binIndex) + ".msp.tsv.gz"
    
    # Read in columns only--> extra column in header that does not match with actual data columns
    rfmixResults_columns = pd.read_csv(pathToRfmixResults + fname , delim_whitespace=True, skiprows=1, nrows=1).columns
    rfmixResults_samples = rfmixResults_columns[7::2] # Get sample ids of individuals in this file

    
    # Rename samples to match genotype data and get columns of samples to get from msp file 
    samples_matched, samples_cho, samples_rfmix_matched_idxs = recodeSamples(sampleMap, admixed_samples_genopheno_matched, rfmixResults_samples)    
    
    rfmixResultsColumnToRead = [i for i in range(6)] + samples_rfmix_matched_idxs # Update column names and get samples to read

    dataypes = getDatatypes(samples_rfmix_matched_idxs)
    # Read in rfmix results (only matched samples) in full
    rfmixResults = pd.read_csv(pathToRfmixResults + fname , delim_whitespace=True,
                               skiprows=2, usecols=rfmixResultsColumnToRead, dtype=dataypes)
    
    # Rename sample ids (from matteo to cholab)
    rfmixResults.columns = ['#chm', 'spos', 'epos', 'sgpos', 'egpos', 'nsnps'] + list(samples_cho)
    
    # Keep dataframe
    rfmixResults_admixedAfricanOnly.append(rfmixResults)
    samples_cho_matched += samples_matched


In [11]:
## Sanity check on rfmix results: only african and european admixed individuals should appear

# Generate counts of haplotypes inherited from each ancestral population
for i in range(len(rfmixResults_admixedAfricanOnly)):
    
    rfmixResults_tractorIn = rfmixResults_admixedAfricanOnly[i]
    
    individual_ancestry_counts = {}
    individual_ancestry_groups = {}
    
    haplotypes = rfmixResults_tractorIn.columns[6:]
    
    # Check at ancestral breakdown for each individual
    for haplotype in haplotypes:
        sample_id = haplotype.split("_")[0]
        if sample_id in individual_ancestry_counts:
            individual_ancestry_counts[sample_id] += rfmixResults_tractorIn[haplotype].value_counts()
        else:
            individual_ancestry_counts[sample_id] = rfmixResults_tractorIn[haplotype].value_counts()

        if sample_id in individual_ancestry_groups:
            groups = individual_ancestry_groups[sample_id]
            individual_ancestry_groups[sample_id] =  groups.union(set(rfmixResults_tractorIn[haplotype].unique())) 
        else:
            individual_ancestry_groups[sample_id] = set(rfmixResults_tractorIn[haplotype].unique())

    eur_code = 3
    afr_code = 0
    amr_code = 1
    sas_code = 4 # should not appear
    eas_code = 2 # should not appear


    samples_with_afr_eur_inherited_tracts = []
    samples_with_afr_only_inherited_tracts = []
    samples_with_eur_only_inherited_tracts = []
    samples_with_amr_inherited_tracts = []
    samples_with_sas_inherited_tracts = []
    samples_with_eas_inherited_tracts = []


    for sample_id, ancestry_groups in individual_ancestry_groups.items():
        if set([0, 3]) == ancestry_groups:
            samples_with_afr_eur_inherited_tracts.append(sample_id)

        if set([0]) == ancestry_groups:
            samples_with_afr_only_inherited_tracts.append(sample_id)

        if set([3]) == ancestry_groups:
            samples_with_eur_only_inherited_tracts.append(sample_id)

    print("AFR + EUR: ", len(samples_with_afr_eur_inherited_tracts))
    print("AFR only: ", len(samples_with_afr_only_inherited_tracts))
    print("EUR only: ", len(samples_with_eur_only_inherited_tracts))
    print(" w/AMR: ", len(samples_with_amr_inherited_tracts))
    print(" w/SAS: ", len(samples_with_sas_inherited_tracts))
    print(" w/EAS: ", len(samples_with_eas_inherited_tracts))

AFR + EUR:  26
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  27
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  22
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  29
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  34
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  24
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  30
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  33
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  19
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  28
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  29
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  35
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  31
AFR only:  0
EUR only:  0
 w/AMR:  0
 w/SAS:  0
 w/EAS:  0
AFR + EUR:  23
AFR only:  0
EUR only: 

In [19]:
# Pickle rfmixResults for reference
with open(outputPath + 'rfmixResults_admixedAfricanOnly.pkl', 'wb') as f:
    pickle.dump(rfmixResults_admixedAfricanOnly, f)

In [20]:
print(len(samples_cho_matched))
print(samples_cho_matched[:5])

2539
[4476280, 3845985, 3725323, 5004315, 4981593]


In [21]:
# Get sample order in VCF files and remove individuals that do not rfmix results
admixed_samples_genopheno_matched_rfmix = [sample for sample in admixed_samples_genopheno_matched if sample in samples_cho_matched]
print(len(samples_cho_matched))
print(admixed_samples_genopheno_matched[:5])
print(len(admixed_samples_genopheno_matched_rfmix))
print(admixed_samples_genopheno_matched_rfmix[:5]) # Use to reorder samples in filtered, and recoded rfmix results

2539
[5432249, 5093858, 3329090, 3640465, 4911551]
2539
[5432249, 5093858, 3329090, 3640465, 4911551]


In [22]:
# Output final list of individuals to text file for subsequent analysis extract (plink)
fname_out = "admixed_samples_chr19_nomiss_genotype_ordered_final.txt"
with open(outputPath + "input_data/" + fname_out, 'w') as f:
    for sample in admixed_samples_genopheno_matched_rfmix:
        sample_string = str(sample) + " " + str(sample)
        f.write(sample_string + '\n')

In [23]:
# Concatenate rfmix results of chromosome together
rfmixResults_admixedAfricanOnly_allSamples = pd.concat(rfmixResults_admixedAfricanOnly)
print(rfmixResults_admixedAfricanOnly_allSamples.shape)
print(rfmixResults_admixedAfricanOnly_allSamples.columns)
print(rfmixResults_admixedAfricanOnly_allSamples.iloc[:3, :10])

(20228, 5084)
Index(['#chm', 'spos', 'epos', 'sgpos', 'egpos', 'nsnps', '4476280_4476280.0',
       '4476280_4476280.1', '3845985_3845985.0', '3845985_3845985.1',
       ...
       '2495681_2495681.0', '2495681_2495681.1', '5487635_5487635.0',
       '5487635_5487635.1', '1948895_1948895.0', '1948895_1948895.1',
       '2294175_2294175.0', '2294175_2294175.1', '4326419_4326419.0',
       '4326419_4326419.1'],
      dtype='object', length=5084)
   #chm    spos     epos  sgpos  egpos  nsnps  4476280_4476280.0  \
0    19  601373   768721   1.59   2.46    100                0.0   
1    19  768721   849618   2.46   3.08     45                0.0   
2    19  849618  1090803   3.08   3.80    140                0.0   

   4476280_4476280.1  3845985_3845985.0  3845985_3845985.1  
0                3.0                0.0                3.0  
1                3.0                0.0                3.0  
2                3.0                0.0                3.0  


In [24]:
# Reorder rfmix results based on order of samples in vcf file
rfmixResults_admixedAfricanOnly_sample_ids = rfmixResults_admixedAfricanOnly_allSamples.columns[6::2]
rfmixResults_admixedAfricanOnly_sample_ids = [int(id.split('_')[0]) for id in rfmixResults_admixedAfricanOnly_sample_ids]
rfmixResults_admixedAfricanOnly_sample_ids[:5]

reorder = {k:v for v,k in enumerate(admixed_samples_genopheno_matched_rfmix)}
rfmixResults_admixedAfricanOnly_sample_ids.sort(key=reorder.get)

print(admixed_samples_genopheno_matched_rfmix[:5])
print(rfmixResults_admixedAfricanOnly_sample_ids[:5])

# Convert sample ids back to rfmix format: sample_sample.0 and  sample_sample.1
rfmixResults_admixedAfricanOnly_sample_ids_reorder = []
for sample in rfmixResults_admixedAfricanOnly_sample_ids:
    sample_string = str(sample) + "_" + str(sample)
    rfmixResults_admixedAfricanOnly_sample_ids_reorder.append(sample_string + ".0")
    rfmixResults_admixedAfricanOnly_sample_ids_reorder.append(sample_string + ".1")

rfmixResults_admixedAfricanOnly_sample_ids_reorder[:5] # Use to reorder columns in rfmix results (concatenated)

[5432249, 5093858, 3329090, 3640465, 4911551]
[5432249, 5093858, 3329090, 3640465, 4911551]


['5432249_5432249.0',
 '5432249_5432249.1',
 '5093858_5093858.0',
 '5093858_5093858.1',
 '3329090_3329090.0']

In [38]:
# Reorder columns of concatenated, filtered, and reordered rfmix results to order of geno
reorderedColumns = list(rfmixResults_admixedAfricanOnly_allSamples.columns[0:6]) + list(rfmixResults_admixedAfricanOnly_sample_ids_reorder)

rfmixResults_admixedAfricanOnly_allSamples_final = rfmixResults_admixedAfricanOnly_allSamples[reorderedColumns]

rfmixResults_admixedAfricanOnly_allSamples_final.shape

(20228, 5084)

In [39]:
# Fill in NA values and recode ancestral codes (AFR = 0, EUR = 3 --> 1)
print(rfmixResults_admixedAfricanOnly_allSamples_final.iloc[:3, :7])
rfmixResults_admixedAfricanOnly_allSamples_final_int = rfmixResults_admixedAfricanOnly_allSamples_final.fillna(-1)
# rfmixResults_admixedAfricanOnly_allSamples_final_int = rfmixResults_admixedAfricanOnly_allSamples_final_int.replace(3, 1)

   #chm    spos     epos  sgpos  egpos  nsnps  5432249_5432249.0
0    19  601373   768721   1.59   2.46    100                NaN
1    19  768721   849618   2.46   3.08     45                NaN
2    19  849618  1090803   3.08   3.80    140                NaN


In [40]:
# Change datypes to match output file format for msp data
rfmixResults_datatypes = dict.fromkeys(rfmixResults_admixedAfricanOnly_allSamples_final_int.columns[6:], np.int8)
rfmixResults_datatypes['#chm'] = np.int8 #chm
rfmixResults_datatypes['spos'] = np.int64 #spos
rfmixResults_datatypes['epos'] = np.int64 #epos
rfmixResults_datatypes['sgpos'] = np.float64 #sgpos
rfmixResults_datatypes['egpos'] = np.float64 #egpos
rfmixResults_datatypes['nsnps'] = np.int64 #nsnps

rfmixResults_admixedAfricanOnly_allSamples_final_int = rfmixResults_admixedAfricanOnly_allSamples_final_int.astype(rfmixResults_datatypes)
print(rfmixResults_admixedAfricanOnly_allSamples_final_int.iloc[:3, :7])

   #chm    spos     epos  sgpos  egpos  nsnps  5432249_5432249.0
0    19  601373   768721   1.59   2.46    100                 -1
1    19  768721   849618   2.46   3.08     45                 -1
2    19  849618  1090803   3.08   3.80    140                 -1


In [41]:
rfmixResults_admixedAfricanOnly_allSamples_final_int.shape

(20228, 5084)

In [42]:
rfmixResults_admixedAfricanOnly_allSamples_final_int

,#chm,spos,epos,sgpos,egpos,nsnps,5432249_5432249.0,5432249_5432249.1,5093858_5093858.0,5093858_5093858.1,...,4563247_4563247.0,4563247_4563247.1,4145550_4145550.0,4145550_4145550.1,1816775_1816775.0,1816775_1816775.1,1571060_1571060.0,1571060_1571060.1,1599017_1599017.0,1599017_1599017.1
0,19,601373,768721,1.59,2.46,100,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,19,768721,849618,2.46,3.08,45,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,19,849618,1090803,3.08,3.80,140,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,19,1090803,1252451,3.80,4.39,50,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,19,1252451,1343212,4.39,4.94,25,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,19,57527351,57674644,105.85,106.20,55,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
214,19,57674644,57784866,106.20,106.36,40,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
215,19,57784866,57946540,106.36,106.57,55,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
216,19,57946540,58537665,106.57,107.07,160,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [37]:
fname_out = outputPath + "input_data/" "pca.chr19.ukb_hap_admixed.mine.msp.tsv"
# Write supplemental information to file
with open(fname_out, 'w') as f:
    f.write('#Subpopulation order/codes: AFR=0       EUR=1\n')
# Write new file out
rfmixResults_admixedAfricanOnly_allSamples_final_int.to_csv(fname_out, sep = "\t", mode='a', index=False)

In [34]:
# Test reading
test = pd.read_csv(fname_out, sep="\t",
                               skiprows=1) 

print(test.shape)
print(len(test.columns[7::2]))
print(test.columns) # Should be 2359 (ie number of individuals)

(20227, 5084)
2539
Index(['19', '601373', '768721', '1.59', '2.46', '100', '-1', '-1.1', '-1.2',
       '-1.3',
       ...
       '-1.5016', '-1.5017', '-1.5018', '-1.5019', '-1.5020', '-1.5021',
       '-1.5022', '-1.5023', '-1.5024', '-1.5025'],
      dtype='object', length=5084)


In [33]:
test.iloc[:10, :9]

,19,601373,768721,1.59,2.46,100,-1,-1.1,-1.2
0,19,768721,849618,2.46,3.08,45,-1,-1,-1
1,19,849618,1090803,3.08,3.80,140,-1,-1,-1
2,19,1090803,1252451,3.80,4.39,50,-1,-1,-1
3,19,1252451,1343212,4.39,4.94,25,-1,-1,-1
4,19,1343212,1550872,4.94,5.65,90,-1,-1,-1
5,19,1550872,1664092,5.65,5.85,25,-1,-1,-1
6,19,1664092,1750447,5.85,6.41,40,-1,-1,-1
7,19,1750447,2014037,6.41,7.24,100,-1,-1,-1
8,19,2014037,2252990,7.24,7.76,80,-1,-1,-1
9,19,2252990,2501255,7.76,8.30,100,-1,-1,-1


In [ ]:
rfmixResults_tractorIn = rfmixResults_admixedAfricanOnly[i]
    
    individual_ancestry_counts = {}
    individual_ancestry_groups = {}
    
    haplotypes = rfmixResults_tractorIn.columns[6:]
    
    # Check at ancestral breakdown for each individual
    for haplotype in haplotypes:
        sample_id = haplotype.split("_")[0]
        if sample_id in individual_ancestry_counts:
            individual_ancestry_counts[sample_id] += rfmixResults_tractorIn[haplotype].value_counts()
        else:
            individual_ancestry_counts[sample_id] = rfmixResults_tractorIn[haplotype].value_counts()

        if sample_id in individual_ancestry_groups:
            groups = individual_ancestry_groups[sample_id]
            individual_ancestry_groups[sample_id] =  groups.union(set(rfmixResults_tractorIn[haplotype].unique())) 
        else:
            individual_ancestry_groups[sample_id] = set(rfmixResults_tractorIn[haplotype].unique())

    eur_code = 3
    afr_code = 0
    amr_code = 1
    sas_code = 4 # should not appear
    eas_code = 2 # should not appear


    samples_with_afr_eur_inherited_tracts = []
    samples_with_afr_only_inherited_tracts = []
    samples_with_eur_only_inherited_tracts = []
    samples_with_amr_inherited_tracts = []
    samples_with_sas_inherited_tracts = []
    samples_with_eas_inherited_tracts = []


    for sample_id, ancestry_groups in individual_ancestry_groups.items():
        if set([0, 3]) == ancestry_groups:
            samples_with_afr_eur_inherited_tracts.append(sample_id)

        if set([0]) == ancestry_groups:
            samples_with_afr_only_inherited_tracts.append(sample_id)

        if set([3]) == ancestry_groups:
            samples_with_eur_only_inherited_tracts.append(sample_id)

    print("AFR + EUR: ", len(samples_with_afr_eur_inherited_tracts))
    print("AFR only: ", len(samples_with_afr_only_inherited_tracts))
    print("EUR only: ", len(samples_with_eur_only_inherited_tracts))
    print(" w/AMR: ", len(samples_with_amr_inherited_tracts))
    print(" w/SAS: ", len(samples_with_sas_inherited_tracts))
    print(" w/EAS: ", len(samples_with_eas_inherited_tracts))